# Set up

In [1]:
using IntervalArithmetic, Combinatorics, Polynomials, Serialization, LinearAlgebra, Plots, LaTeXStrings

In [ ]:
include("quadrature.jl")

We aim to find a self-similar solution to the viscous Burger's equation on $\mathbb{R}_+$

$$\partial_t v + v^2\partial_x v = \Delta v$$

The profile for 
$$\mathcal{L}u-\frac{u}{4}+u^2\frac{\partial u}{\partial x} = 0$$

The Vandermonde matrices below have been adapted to treat the above nonlinearity. The quadratures are still rigorous.

In [2]:
n = 1500

# approximate solution
ū = interval.(deserialize("ubar"));

# regularised Vandermonde matrices for 2p and p+1 products
V̄4 = deserialize("V4r");
V̄6 = deserialize("V6r");
DV̄4 = deserialize("DV4r");
DV̄6 = deserialize("DV6r");

p = interval(3)
d = 1
α = d//2

# m = n+1, (n+1)ᵗʰ eigenvalue
λₘ = interval(d//2+n+1);
λ₀ = interval(d//2);

# generator
𝔏 = Diagonal(interval.(collect(0:n)).+λ₀);

In [3]:
LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))
function doublefac(n)
    if n==-1
        return 1
    else
        return doublefactorial(big(n))
    end
end

# computes Lᵖ/H¹ norms before taking fractional exponent
function L2(u)
    return sum(u.^2)
end

function H2(u)
    return sum((𝔏*u).^2)
end

function L6(u)
    return sum((V̄6*u).^4 .* (DV̄6*u).^2)
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        all(isguaranteed.(A)) || error("matrix not guaranteed")
        return sqrt(interval(maximum(sup.(sum(abs.(A), dims = 1))))*interval(maximum(sup.(sum(abs.(A), dims = 2)))))
    end
end

op_norm (generic function with 1 method)

# Proof

In [4]:
# Gram matrix induced by ū∂ₓū
G = V̄4'*(((V̄4*ū).*(DV̄4*ū)).*V̄4);
# Non-Gram matrix induced by ū²∂ₓ
DG = V̄4'*(((V̄4*ū).^2).*DV̄4);

In [5]:
# PₙF(ū)
PFū = ū + 𝔏\(-ū/interval(4)+V̄4'*((V̄4*ū).^2 .*(DV̄4*ū)));
println(norm(PFū))

Interval{Float64}(1.4201949660419926e-11, 1.4735509426955604e-11, com)


In [6]:
# PₙDF(ū)Pₙ
DFū = interval.(I(n+1)) - inv(𝔏)/interval(4) + 𝔏\(interval(2)*G+DG);
# approximate numerical inverse of PₙDF(ū)Pₙ
Aₙ = interval.(inv(mid.(DFū)));

In [7]:
# rigorous L²-norms of ūψ̂ⱼ∂ₓū
int = sum((((V̄6*ū).*(DV̄6*ū)).*V̄6).^2, dims = 1);
# rigorous L²-norms of ū²∂ₓψ̂ⱼ
Dint = sum((((V̄6*ū).^2).*DV̄6).^2, dims = 1);

In [8]:
w = interval(2)*sqrt.([int[i] - L2(G[:,i]) for i=1:n+1])+sqrt.([Dint[i] - L2(DG[:,i]) for i=1:n+1]);

In [9]:
Y = sqrt(H2(Aₙ*PFū)+(L6(ū)-L2(V̄4'*((V̄4*ū).^2 .*(DV̄4*ū)))))

[0.000756363, 0.000756364]_com

In [10]:
Z¹¹ = interval(op_norm(𝔏*(interval.(I(n+1)) - Aₙ*DFū)*inv(𝔏)))

[2.89474e-11, 2.89475e-11]_com

In [11]:
Z²¹ = norm(𝔏\w)

[0.0259465, 0.0259466]_com

In [12]:
ψ̂₀ = interval.(Float64, sqrt.([interval(BigFloat,doublefac(2*m-1)//2^m//factorial(m)) for m=0:big(n)]/sqrt(interval(BigFloat,π))));
# sup∂ₓψ̂ = (supψ̂ + 2*sqrt.(1 .- 0.5 ./interval.(1:n+1)).*supψ̂)./sqrt(interval(2)*exp(interval(1)));

In [13]:
supψ̂ = [interval(BigFloat, (2^(2*k+1)- binomial(2*k, k))*factorial(k)//2^k)/sqrt(interval(BigFloat, factorial(2*k))) for k=0:big(n)]/interval(BigFloat, π)^interval(1//4);

In [14]:
sup∂ₓψ̂ = [sqrt(interval((2*k+1)*factorial(2*k+1)//4^k))/interval(factorial(k))*exp(interval(big(-0.5))) for k=0:big(n)]/interval(BigFloat, π)^interval(1//4);

In [15]:
supū = interval(Float64, sum(abs.(ū).*supψ̂))

[1.66592, 1.66593]_com

In [16]:
sup∂ₓū = interval(Float64, sum(abs.(ū).*sup∂ₓψ̂))

[1.18934, 1.18935]_com

In [17]:
Sₙ = (interval(pi)*log(interval(4)) - sum([interval(BigFloat, doublefac(2*m-1)//(m+1//2)^2//factorial(m)//2^m) for m=0:big(n)]))/
sqrt(interval(π))

[3.64929e-06, 3.6493e-06]₂₅₆_com

In [18]:
ū₀ = sum(ū.*ψ̂₀)

[1.16537, 1.16538]_com

In [19]:
Dint = sum((((V̄6*ū).^2).*(DV̄6+V̄6)).^2, dims = 1)

1×1501 Matrix{Interval{Float64}}:
 [0.0, 2.81907e-33]_com  [0.198926, 0.198928]_com  …  [11.2245, 11.2246]_com

In [20]:
DG = V̄4'*(((V̄4*ū).^2).*(V̄4+DV̄4));

In [21]:
w̃ = ū₀^2*sqrt(Sₙ)*ψ̂₀+sqrt.(abs.([Dint[i] - L2(DG[:,i]) for i=1:n+1]))/λₘ;

In [22]:
Z¹² = norm(abs.(𝔏*interval.(Aₙ)*inv(𝔏))*w̃)

[0.0339152, 0.0339153]₂₅₆_com

In [23]:
Z²² = (interval(1//4)+interval(2)*supū*sup∂ₓū)/λₘ + supū^2/sqrt(λₘ)

[0.0744275, 0.0744276]_com

In [24]:
Z₁ = op_norm([Z¹¹ Z¹² ; Z²¹ Z²²])

[0.0851833, 0.0851835]₂₅₆_com

In [25]:
[Z¹¹ Z¹² ; Z²¹ Z²²]

2×2 Matrix{Interval{BigFloat}}:
 [2.89474e-11, 2.89475e-11]₂₅₆_com  [0.0339152, 0.0339153]₂₅₆_com
 [0.0259465, 0.0259466]₂₅₆_com      [0.0744275, 0.0744276]₂₅₆_com

In [26]:
op_n = interval(max(op_norm(𝔏*Aₙ*inv(𝔏)), interval(1)))

[5.79664, 5.79665]_com

In [27]:
Z₂ = interval(2)^interval(15//4)*op_n*(sqrt(interval(2))*supū+sup∂ₓū)

[276.498, 276.499]_com

In [28]:
Z₃ = interval(96)*op_n

[556.477, 556.479]_com

In [29]:
δ̄ = (-Z₂+sqrt(Z₂^2 +interval(2)*Z₃-interval(2)*Z₁*Z₃))/Z₃

[0.00329762, 0.00329764]₂₅₆_com

In [30]:
P(δ) = Z₃/interval(6)*δ^3 + Z₂/interval(2)*δ^2 - (interval(1) - Z₁)*δ + Y

P (generic function with 1 method)

In [31]:
δ̲ = Y/(interval(1)-Z₁)*(interval(1.17163))
if sup(P(δ̲))<0
    println("δ̲ is validated")
end

δ̲ is validated


# Check positivity (under regularity assumption)

In [32]:
# infinity nor error
inf_err = interval(2)^interval(7//4)*δ̲

[0.00325828, 0.0032583]₂₅₆_com

In [33]:
m = 10
supm = inf_err+sum(abs.((supψ̂.*ū)[m+2:end]))
Pₘ = zeros(Interval{BigFloat},m+1)
for j = 0:m
#     println(j)
    Lⱼ =  [interval((-1)^i)*interval(BigFloat, binomial(j,i)//factorial(i)) for i=0:j]
    Pₘ[1:j+1] += ū[Int64(j+1)]*Lⱼ
end

In [34]:
r₀ = interval(BigFloat,4.0)
z = mince(interval(0,r₀.^2/interval(4)), 2000)

if all(inf.(evalpoly.(z,tuple(Pₘ)).*exp.(-z).-supm.*exp.(-z/interval(2))).>0) && inf(r₀^2/interval(16)-interval(4)*r₀*((supū+inf_err)*exp(-r₀^2/interval(8)))^2)>0
    println("positivity checked")
else
    println("positivity not checked")
end

positivity checked
